In [1]:
import os
os.environ['PYSPARK_PYTHON'] = '/Users/sheethalsupriya/Documents/python_env/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/Users/sheethalsupriya/Documents/python_env/bin/python3'


In [2]:
pip install pyspark



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Flight Delay Analysis") \
    .master("local[*]") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/09 17:36:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("hdfs://localhost:9000/user/ssn/airline_data/*.csv")
df.show(5)
df.printSchema()


25/05/09 17:36:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+-------+-----+----------+---------+----------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+------------+---------+-------------+-------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+----------------+------------+--------------+----------------+----

25/05/09 17:36:52 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Year, Quarter, Month, DayofMonth, DayOfWeek, FlightDate, Reporting_Airline, DOT_ID_Reporting_Airline, IATA_CODE_Reporting_Airline, Tail_Number, Flight_Number_Reporting_Airline, OriginAirportID, OriginAirportSeqID, OriginCityMarketID, Origin, OriginCityName, OriginState, OriginStateFips, OriginStateName, OriginWac, DestAirportID, DestAirportSeqID, DestCityMarketID, Dest, DestCityName, DestState, DestStateFips, DestStateName, DestWac, CRSDepTime, DepTime, DepDelay, DepDelayMinutes, DepDel15, DepartureDelayGroups, DepTimeBlk, TaxiOut, WheelsOff, WheelsOn, TaxiIn, CRSArrTime, ArrTime, ArrDelay, ArrDelayMinutes, ArrDel15, ArrivalDelayGroups, ArrTimeBlk, Cancelled, CancellationCode, Diverted, CRSElapsedTime, ActualElapsedTime, AirTime, Flights, Distance, DistanceGroup, CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay, FirstDepTime, TotalAddGTime, LongestAddGTime, DivAirportLandi

### Average Departure and Arrival Delays by Day of Week

In [5]:
from pyspark.sql.functions import avg

print(" Average Departure and Arrival Delays by Day of Week:")
df.groupBy("DayOfWeek") \
  .agg(
      avg("DepDelay").alias("AvgDepDelay"),
      avg("ArrDelay").alias("AvgArrDelay")
  ).orderBy("DayOfWeek").show()



 Average Departure and Arrival Delays by Day of Week:


+---------+------------------+-----------------+
|DayOfWeek|       AvgDepDelay|      AvgArrDelay|
+---------+------------------+-----------------+
|        1|12.732981348582861|7.025029487104243|
|        2|10.461864081536971| 4.39359769421264|
|        3|10.415939915930556|4.825223011256957|
|        4|13.155883724303209|8.178800868027931|
|        5|14.662681586274218|9.607454169575464|
|        6|12.589481874313925|6.453122999593656|
|        7|14.393895904811984|8.774946156229994|
+---------+------------------+-----------------+



### On-Time Performance by Airline


In [7]:
print("=== On-Time Performance by Airline (Percentage Delayed) ===")
df.groupBy("Reporting_Airline") \
  .agg((avg("DepDel15") * 100).alias("PercentDelayed")) \
  .orderBy("PercentDelayed", ascending=False).show()


=== On-Time Performance by Airline (Percentage Delayed) ===


+-----------------+------------------+
|Reporting_Airline|    PercentDelayed|
+-----------------+------------------+
|               F9|27.714623254263543|
|               B6|25.813607592262382|
|               AA|25.612785921213167|
|               NK|24.986245566390018|
|               WN|23.374933501523874|
|               G4|21.248231628464058|
|               AS|20.125457613486073|
|               OH|19.807406945292342|
|               UA|19.026934726811405|
|               MQ|18.770980065542904|
|               DL|17.169085448517738|
|               OO|17.152771175967295|
|               9E|14.923573031066503|
|               HA|13.306404929441578|
|               YX| 11.72645694492272|
+-----------------+------------------+



### Cancellation Rate by Origin Airport

In [8]:
print("=== Cancellation Rate by Origin Airport ===")
df.groupBy("Origin") \
  .agg((avg("Cancelled") * 100).alias("CancelRate")) \
  .orderBy("CancelRate", ascending=False).show()


=== Cancellation Rate by Origin Airport ===


+------+------------------+
|Origin|        CancelRate|
+------+------------------+
|   EWN|7.8431372549019605|
|   ADK|7.6923076923076925|
|   BIH| 6.934306569343065|
|   GST| 6.666666666666667|
|   ASE|  6.54158215010142|
|   EAU| 6.451612903225806|
|   CNY| 6.451612903225806|
|   OTZ|6.0109289617486334|
|   JMS| 5.691056910569105|
|   ACK| 5.492537313432836|
|   BRW|  5.21978021978022|
|   MVY| 4.837209302325581|
|   DVL| 4.277286135693215|
|   CMX|  4.10958904109589|
|   PBG| 3.992015968063872|
|   MCW|3.6682615629984054|
|   LAR|3.6682615629984054|
|   OME|3.5616438356164384|
|   LBL|  3.53356890459364|
|   SRQ| 3.524473797913709|
+------+------------------+
only showing top 20 rows



### Busiest Airports by Number of Departures

In [9]:
print("=== Busiest Airports (by Departure Count) ===")
df.groupBy("Origin") \
  .count() \
  .orderBy("count", ascending=False) \
  .show()


=== Busiest Airports (by Departure Count) ===


+------+------+
|Origin| count|
+------+------+
|   ATL|341910|
|   DFW|313582|
|   DEN|308645|
|   ORD|280052|
|   CLT|217555|
|   LAX|194053|
|   PHX|193551|
|   LAS|189252|
|   SEA|163724|
|   LGA|162432|
|   MCO|159553|
|   BOS|143490|
|   DCA|140016|
|   SFO|134978|
|   DTW|131367|
|   EWR|128979|
|   MSP|126779|
|   JFK|123348|
|   IAH|115940|
|   SLC|113247|
+------+------+
only showing top 20 rows



### Hourly Delay Trends

In [10]:
print("=== Average Departure Delay by Time Block ===")
df.groupBy("DepTimeBlk") \
  .agg(avg("DepDelay").alias("AvgDelay")) \
  .orderBy("DepTimeBlk") \
  .show()


=== Average Departure Delay by Time Block ===


+----------+------------------+
|DepTimeBlk|          AvgDelay|
+----------+------------------+
| 0001-0559| 6.031159056004206|
| 0600-0659| 4.012741915358007|
| 0700-0759| 4.915185412825241|
| 0800-0859|5.8123940817767235|
| 0900-0959|7.2120234012507565|
| 1000-1059| 8.692152857333463|
| 1100-1159| 9.910065399503678|
| 1200-1259|11.455687940597732|
| 1300-1359|12.918527332372836|
| 1400-1459|14.583383139587006|
| 1500-1559|15.885922541070121|
| 1600-1659|17.315141657037724|
| 1700-1759|18.022344182342078|
| 1800-1859|19.832197808105196|
| 1900-1959|20.771263782688784|
| 2000-2059|21.345682217350614|
| 2100-2159| 20.41499668961578|
| 2200-2259|19.852344343902498|
| 2300-2359|15.426155580608794|
+----------+------------------+



### Top Delayed Routes

In [12]:

from pyspark.sql.functions import concat_ws

print("=== Top Delayed Routes by Average Arrival Delay ===")
df.withColumn("Route", concat_ws(" -> ", "Origin", "Dest")) \
  .groupBy("Route") \
  .agg(avg("ArrDelay").alias("AvgArrDelay")) \
  .orderBy("AvgArrDelay", ascending=False) \
  .show(10)


=== Top Delayed Routes by Average Arrival Delay ===


+----------+-----------------+
|     Route|      AvgArrDelay|
+----------+-----------------+
|JFK -> LGA|            755.0|
|SDF -> SLC|            365.0|
|IAD -> MSN|            325.0|
|AVL -> USA|            310.0|
|SRQ -> IAH|            299.6|
|IAH -> SDF|            194.0|
|MVY -> CLT|190.0909090909091|
|PIE -> VPS|            178.0|
|OKC -> CAE|            171.0|
|LAX -> ATW|            164.0|
+----------+-----------------+
only showing top 10 rows

